<a href="https://colab.research.google.com/github/seinJOO/PythonClass/blob/main/Python_Basic/0602/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 케라스를 이용한 mnist 테스트 

In [ ]:
# 필요한 모듈 import
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

In [ ]:
# MNIST 데이터 읽기
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# x는 이차배열 형태의 픽셀값, y에는 해당 x값이 나타내는 레이블(숫자)이 있음

# 데이터를 float32 자료형으로 변환하고 정규화 처리
x_train = x_train.reshape(60000, 784).astype('float32')    # 픽셀데이터를 28x28로 reshape / reshape() : 입력을 주어진 모양으로 재구성
x_test = x_test.reshape(10000, 784).astype('float32')
x_train[0]  # 여기서 보이는 숫자는 색상값임!


In [ ]:
import matplotlib.pyplot as plt
plt.show(x_train[0])    # 이미지 확인가능

In [ ]:
x_train /= 255
x_test /= 255
x_train[0]      # << 색상값은 255까지니까 나눠서 1미만의 숫자로 정규화 처리

In [ ]:
# 레이블 데이터를 0-9까지 카테고리를 나타내는 배열로 변환 
# (정답인 숫자를 바이너리처럼 변환 - 정답에 해당하는 인덱스는 1, 나머지는 0)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [ ]:
### 모델 구조 정의하기
model = Sequential()  # 정확히 하나의 입력 텐서와 하나의 출력 텐서가 있는 일반 레이어 스택

# 활성화 함수 Activation() : 딥러닝 네트워크에서 노드에 입력된 값들을 비선형 함수에 통과시킨 후 다음 레이어로 전달할 때 사용하는 함수

# 점진적 빌드 시작
model.add(Dense(512, input_shape=(784,)))   # Dense : 뉴런의 입력과 출력을 연결 (출력뉴런수, input_shape=입력값개수)
model.add(Activation('relu'))   # Activation() : 활성화함수를 설정
model.add(Dropout(0.2))   # 레이어의 모든 노드에서 나가는 activation을 특정 확률로 지워버림 - 뉴럴 네트워크의 학습데이터 과적합 방지
model.add(Dense(512))
model.add(Activation('relu')) # relu : 경사함수 (가장 많이 사용되는 활성화 함수)
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))  # softmax : 다중 클래스 분류 문제를 위한 소프트맥스 회귀


# 모델 구축하기
model.compile(
    loss = 'categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy'])

# 데이터 훈련하기
hist = model.fit(x_train, y_train)

# 테스트 데이터 평가
score = model.evaluate(x_test, y_test, verbose=1)
print("loss =", score[0])
print("score =", score[1])


313/313 [==============================] - 1s 4ms/step - loss: 0.0931 - accuracy: 0.9701
loss = 0.09308817982673645
score = 0.9700999855995178


### Keras로 비만도 판정하기

In [ ]:
# 필요한 모듈 import
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping
import pandas as pd, numpy as np

In [ ]:
# BMI 데이터를 읽어서 정규화하기
csv = pd.read_csv('/content/sample_data/bmi.csv') # 20000 entries

# 몸무게와 키 데이터 정규화
csv['weight'] /= 100    # 몸무게 데이터는 100kg 미만임
csv['height'] /= 200    # 키 데이터는 2m 미만임
X = csv[['weight','height']].to_numpy()

# 레이블
bclass = {'thin' : [1,0,0], 'normal' : [0,1,0], 'fat' : [0,0,1]}
Y = np.empty((20000,3))
for i, v in enumerate(csv['label']) :
  Y[i] = bclass[v]

# 훈련 전용 데이터와 테스트 전용 데이터로 나누기
X_train, Y_train = X[0:15000], Y[0:15000]
X_test, Y_test = X[15000:20001], Y[15000:20001]

# 모델 구조 정의하기
model = Sequential()
model.add(Dense(512, input_shape=(2,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(3))
model.add(Activation('softmax'))

# 모델 구축하기
model.compile(
    loss = 'categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

# 데이터 훈련하기
hist = model.fit(
    X_train, Y_train,
    batch_size = 100, # 튜닝값1
    epochs = 20,      # 회차(튜닝값2 ) - 회차의 기본값은 1
    validation_split=0.1,
    # callbacks = [EarlyStopping(monitor='val_loss', patience=2)],  
                # 데이터 정밀도에 문제가 생길 경우 훈련 중지 - 의미없는 데이터로 인한 시간낭비 방지
    verbose = 1)

# 테스트 데이터 평가
score = model.evaluate(X_test, Y_test)
print("loss =", score[0])
print("score =", score[1])



Epoch 1/20
135/135 [==============================] - 2s 12ms/step - loss: 0.5215 - accuracy: 0.7867 - val_loss: 0.2765 - val_accuracy: 0.9093
Epoch 2/20
135/135 [==============================] - 1s 11ms/step - loss: 0.2505 - accuracy: 0.9011 - val_loss: 0.3449 - val_accuracy: 0.8147
Epoch 3/20
135/135 [==============================] - 1s 10ms/step - loss: 0.1883 - accuracy: 0.9221 - val_loss: 0.3390 - val_accuracy: 0.8233
Epoch 4/20
135/135 [==============================] - 1s 11ms/step - loss: 0.1613 - accuracy: 0.9332 - val_loss: 0.0966 - val_accuracy: 0.9880
Epoch 5/20
135/135 [==============================] - 2s 11ms/step - loss: 0.1568 - accuracy: 0.9287 - val_loss: 0.0882 - val_accuracy: 0.9773
Epoch 6/20
135/135 [==============================] - 1s 11ms/step - loss: 0.1421 - accuracy: 0.9384 - val_loss: 0.0911 - val_accuracy: 0.9660
Epoch 7/20
135/135 [==============================] - 1s 10ms/step - loss: 0.1357 - accuracy: 0.9446 - val_loss: 0.0823 - val_accuracy: 0.9673